# Случайные леса
__Суммарное количество баллов: 10__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier

In [2]:
from task import gini, entropy, gain

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [3]:
from task import DecisionTree

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [4]:
from task import RandomForestClassifier

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [5]:
from task import feature_importance


def most_important_features(importance, names, k=20):
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [6]:
% load_ext autoreload
% autoreload 2

def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3,
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)


X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

/Users/dev.daniil.bakushkin/Desktop/ML/Practices/hw_9 [Random Forest]/dev/task.py:207: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  predictions = stats.mode(predictions, axis=1)[0].reshape(-1)


Accuracy: 1.0
Importance: [0.001814678091590638, 0.0012539244413147799, 0.16793008641519314, 0.1472846195648635, 0.32104976412246156, -0.0013203271999341348]


### Задание 4 (3 балла)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

Оценка:
1. 1 балл за исправно работающий код
2. +1 балл за точность предсказания возростной группы выше 65%
3. +1 балл за точность предсказания пола выше 75%

In [7]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]

    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [8]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [74]:
from task import rfc_age

rfc_age.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc_age.predict(X_test) == y_age_test))

Accuracy: 0.6923076923076923
Most important features:


/Users/dev.daniil.bakushkin/Desktop/ML/Practices/hw_9 [Random Forest]/dev/task.py:207: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  predictions = stats.mode(predictions, axis=1)[0].reshape(-1)


#### Пол

In [75]:
from task import rfc_gender

rfc_gender = RandomForestClassifier(n_estimators=10)
rfc_gender.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc_gender.predict(X_test) == y_sex_test))

Accuracy: 0.8436317780580076


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [54]:
X, y = synthetic_dataset(1000)

model = CatBoostClassifier(iterations=100,
                           depth=6,
                           learning_rate=0.3,
                           loss_function='MultiClass',
                           verbose=False)

model.fit(X, y)
preds = model.predict(X)

print("Accuracy:", np.mean(preds == y))

Accuracy: 0.333334


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

Оценка:
1. 1 балл за исправно работающий код
2. +1 балл за точность предсказания возростной группы выше 65%
3. +1 балл за точность предсказания пола выше 75%

In [37]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train,
                                                                                     train_size=0.8)

#### Возраст

In [71]:
model = CatBoostClassifier(iterations=50,
                           depth=10,
                           learning_rate=0.5,
                           loss_function='MultiClass',
                           verbose=True)

model.fit(X_train, y_sex_train)
preds = model.predict(X_test)
model.save_model('gender_model.pth')

print("Accuracy:", np.mean(preds.flatten() == y_sex_test))

0:	learn: 0.5782867	total: 61.3ms	remaining: 3s
1:	learn: 0.5202976	total: 294ms	remaining: 7.06s
2:	learn: 0.4731461	total: 364ms	remaining: 5.71s
3:	learn: 0.4454966	total: 449ms	remaining: 5.17s
4:	learn: 0.4200515	total: 527ms	remaining: 4.74s
5:	learn: 0.4013925	total: 605ms	remaining: 4.43s
6:	learn: 0.3877014	total: 684ms	remaining: 4.2s
7:	learn: 0.3732214	total: 760ms	remaining: 3.99s
8:	learn: 0.3644366	total: 845ms	remaining: 3.85s
9:	learn: 0.3529011	total: 919ms	remaining: 3.68s
10:	learn: 0.3442580	total: 1.01s	remaining: 3.57s
11:	learn: 0.3366976	total: 1.1s	remaining: 3.47s
12:	learn: 0.3301906	total: 1.18s	remaining: 3.35s
13:	learn: 0.3242020	total: 1.25s	remaining: 3.21s
14:	learn: 0.3191568	total: 1.32s	remaining: 3.09s
15:	learn: 0.3144719	total: 1.42s	remaining: 3.02s
16:	learn: 0.3074755	total: 1.49s	remaining: 2.9s
17:	learn: 0.3024600	total: 1.56s	remaining: 2.77s
18:	learn: 0.2982452	total: 1.62s	remaining: 2.64s
19:	learn: 0.2943172	total: 1.69s	remaining: 2

#### Пол

In [73]:
model = CatBoostClassifier(iterations=50,
                           depth=10,
                           learning_rate=0.5,
                           loss_function='MultiClass',
                           verbose=True)

model.fit(X_train, y_age_train)
preds = model.predict(X_test)
model.save_model('age_model.pth')

print("Accuracy:", np.mean(preds.flatten() == y_age_test))

0:	learn: 0.9430487	total: 116ms	remaining: 5.7s
1:	learn: 0.8600838	total: 364ms	remaining: 8.75s
2:	learn: 0.8099983	total: 477ms	remaining: 7.46s
3:	learn: 0.7655609	total: 572ms	remaining: 6.58s
4:	learn: 0.7340628	total: 668ms	remaining: 6.01s
5:	learn: 0.7080299	total: 760ms	remaining: 5.58s
6:	learn: 0.6856652	total: 854ms	remaining: 5.24s
7:	learn: 0.6621763	total: 947ms	remaining: 4.97s
8:	learn: 0.6434215	total: 1.04s	remaining: 4.73s
9:	learn: 0.6284765	total: 1.13s	remaining: 4.52s
10:	learn: 0.6149471	total: 1.22s	remaining: 4.33s
11:	learn: 0.6043125	total: 1.31s	remaining: 4.17s
12:	learn: 0.5943439	total: 1.41s	remaining: 4.02s
13:	learn: 0.5827502	total: 1.51s	remaining: 3.87s
14:	learn: 0.5734437	total: 1.6s	remaining: 3.74s
15:	learn: 0.5626220	total: 1.69s	remaining: 3.6s
16:	learn: 0.5526726	total: 1.78s	remaining: 3.46s
17:	learn: 0.5457675	total: 1.87s	remaining: 3.33s
18:	learn: 0.5367592	total: 1.96s	remaining: 3.21s
19:	learn: 0.5271919	total: 2.06s	remaining: